In [1]:
require 'torchnet'
require 'image'

In [2]:
local dir = io.popen("ls ../data/sample/")

lines = {}
for line in dir:lines() do
    lines[#lines + 1] = line
end

eye_sample_data = {}
for i=1, #lines do
    track, eye, file_extension = lines[i]:match("([^_]+)_([^.]+).([^,]+)")
    index = tonumber(track)
    if eye == "left" then
        eyetype = 1
    else
        eyetype = 2
    end
    if eye_sample_data[index] == nil then
        eye_sample_data[index] = {}
    end
    eye_sample_data[index][eyetype] = 1
end

In [3]:
file = io.open('../data/sample_crop_data.txt', "rb")

lines = {}
for line in file:lines() do
    lines[#lines + 1] = line
end

In [10]:
eye_crop_data = {}
for i=1, #lines do
    track, eye, extension, y0, x0, y1, x1 = lines[i]:match("([^,]+)_([^.]+).([^,]+),([^,]+),([^,]+),([^,]+),([^,]+)")
    index = tonumber(track)
    y0, x0, y1, x1 = tonumber(y0), tonumber(x0), tonumber(y1), tonumber(x1)
    if eye == 'left' then
        eyetype = 1
    else
        eyetype = 2
    end
    if eye_crop_data[index] == nil then
        eye_crop_data[index] = {}
    end
    eye_crop_data[index][eyetype] = {x0, y0, x1, y1}
end

In [13]:
length = 0
for k, v in pairs(eye_sample_data) do
    length = length+1
end

In [14]:
-- Index 1 --File track
-- Index 2 --Left or Right Eye
-- Index 3 --Pairing Eye Index
-- Index 4 --Cropping X0
-- Index 5 --Cropping Y0
-- Index 6 --Cropping X1
-- Index 7 --Cropping Y1

eye_sample_tensor = torch.LongTensor(length*2, 7)

In [15]:
index = 1
for k, v in pairs(eye_sample_data) do
    -- Left eye
    eye_sample_tensor[index][1] = k
    eye_sample_tensor[index][2] = 1
    eye_sample_tensor[index][3] = index+1
    
    --Dont'have the cropping information yet
    eye_sample_tensor[index][4] = eye_crop_data[k][1][1]
    eye_sample_tensor[index][5] = eye_crop_data[k][1][2]
    eye_sample_tensor[index][6] = eye_crop_data[k][1][3]
    eye_sample_tensor[index][7] = eye_crop_data[k][1][4]
    index = index + 1
    
    -- Right eye
    eye_sample_tensor[index][1] = k
    eye_sample_tensor[index][2] = 2
    eye_sample_tensor[index][3] = index - 1
    
    --Don't have the cropping information yet
    eye_sample_tensor[index][4] = eye_crop_data[k][2][1]
    eye_sample_tensor[index][5] = eye_crop_data[k][2][2]
    eye_sample_tensor[index][6] = eye_crop_data[k][2][3]
    eye_sample_tensor[index][7] = eye_crop_data[k][2][4]
    index = index + 1
end

In [16]:
eye_sample_tensor

   13     1     2   282     0  2310  1944
   13     2     1   282     0  2310  1944
   17     1     4   705   101  3174  2563
   17     2     3   705   101  3174  2563
   16     1     6   318     0  3600  2592
   16     2     5   324     0  3576  2592
   10     1     8   855   135  3865  3132
   10     2     7   858   137  3865  3131
   15     1    10   860    45  4049  3234
   15     2     9   860    46  4050  3234
[torch.LongTensor of size 10x7]



In [17]:
torch.save('../data/sample_full_metadata.t7', eye_sample_tensor)